# Packages Import

In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from utils.labels import map_class_to_rgb, adjust_mask
import segmentation_models as sm
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from utils.util import *
from arch.deeplabv3plus import Deeplabv3



# Paths and Constants

In [ ]:
##### paths


base_data_dir = 'data'
saved_models = 'checkpoint'

train_path = os.path.join(base_data_dir, 'data')
test_path = os.path.join(base_data_dir, 'testing')

classes_path = os.path.join(base_data_dir, 'classes.csv')


iti_data_dir = os.path.join(base_data_dir, 'aroundITI')
single_frames_dir = os.path.join(base_data_dir, 'single_frames')
videos_dir = os.path.join(base_data_dir, 'videos')

output_images_dir = os.path.join(base_data_dir, 'output_images')
output_videos_dir = os.path.join(base_data_dir, 'output_videos')



## parmeters

batch_sz =8
num_classes = 31
DIM=256
seed = 1







# Read Labels file

In [ ]:
classes = pd.read_csv(classes_path,index_col='name')

# data generators

In [ ]:
# Data generator
train_generator_fn, val_generator_fn, train_length, validation_length = train_valid_generator(train_path, seed, batch_sz, DIM)

# Model (change here)

In [ ]:
sm.set_framework('tf.keras')

model = sm.Unet('inceptionresnetv2', classes=31, activation='softmax')

In [ ]:
#model.summary()

# freeze layers (optional)

In [ ]:
## xception exit_flow_block2_separable_conv1_depthwise
## mobilenetv2 expanded_conv_16_project

# freeze_model_layers(model,'exit_flow_block2_separable_conv1_depthwise')

# model.summary()

# model compilation and fitting (you may change here if needed)

In [ ]:
from tensorflow.python.keras.optimizers import adam_v2

model_name=model._name

cp_name = os.path.join(saved_models,
                            model_name+'_epoch_{epoch:03d}_valLoss_{val_loss:.4f}.h5')

model_checkpoint = ModelCheckpoint(filepath=cp_name
                                   , monitor='val_loss',verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(
    monitor = "val_loss", 
    patience = 20,
    verbose = 1, 
    factor = 0.2, 
    min_lr = 0.000001,
)


model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate=1e-4),
    # optimizer='Nadam',
    loss=sm.losses.cce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
    
)

history = model.fit(
    train_generator_fn(),
    steps_per_epoch=train_length // batch_sz
  , validation_steps=validation_length // batch_sz,
   epochs=100,
   validation_data= val_generator_fn(),  
   callbacks=[model_checkpoint,reduce_lr]
)




# plotting loss and MeanIou

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
iou_score = history.history['iou_score']
val_iou_score = history.history['val_iou_score']

epochs = range(len(loss))

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.plot(epochs, iou_score, 'r', label='Training iou_score')
plt.plot(epochs, val_iou_score, 'b', label='Validation iou_score')
plt.title('Training and validation iou_score')
plt.legend()

plt.show()

# loading Created Model

In [ ]:
from keras.models import load_model

model = load_model(filepath=model_checkpoint._write_filepath, custom_objects={
    "categorical_crossentropy_plus_jaccard_loss":sm.losses.cce_jaccard_loss
    ,"iou_score":sm.metrics.iou_score})

# predicting Validation data

In [ ]:
plot_predictions (train_path,model,DIM,mode='valid')

# predicting test data

In [ ]:
plot_predictions (test_path,model,DIM,mode='testing',test_sample_size=30)

# predicting ITI Data

In [ ]:
plot_predictions (iti_data_dir,model,DIM,mode='ITI')